# Prepare data label for CNN model

## Evironment setup

In [7]:
# Setup 

DATA_PATH = 'data/'
IMAGES_PATH = 'data/formula_images/'

FORMULAS = DATA_PATH + 'im2latex_formulas.lst'

In [8]:
# Import

from  sys import stdout
import numpy as np
import cPickle as pickle
import nltk
import scipy.misc
import scipy.sparse

from tokenizer import Tokenizer

## Label with formula tokenization
目前將'command' token直接載入，將'text' token以character方式載入，還未做過濾，有些'command'與'text'不會顯示在圖片上。

In [9]:
# build formula list

with open(FORMULAS) as formulas_file:
    formula_list = []
    for line in formulas_file:
        formula_list.append(line)

In [10]:
# tokenize formula list

formula_tokens_list = []

for formula in formula_list:
    tokens_dict = {}
    tokenizer = Tokenizer(formula)
    for token in tokenizer.tokenize():
        # formation of token: (line_number, type, token_part, token_full)        
        # command token process
        if token[1] == 'command':
            if '\\'+token[2] not in tokens_dict:
                tokens_dict['\\'+token[2]] = 1
            else:
                tokens_dict['\\'+token[2]] += 1
        # text token process
        elif token[1] == 'text':
            for c in token[3]:
                if c not in tokens_dict:
                    tokens_dict[c] = 1
                else:
                    tokens_dict[c] += 1
    formula_tokens_list.append(tokens_dict)

In [5]:
# build token dictionary

tokens_count = {}
tokens_dict = {}

# count token frequency
for formula_tokens in formula_tokens_list:
    for token, value in formula_tokens.iteritems():
        if token not in tokens_count:
            tokens_count[token] = value
        else:
            tokens_count[token] += 1

#build token dictionary according to token frequency
index = 2 # 0 for '<eos>' 1 for 'UNK'
tokens_sorted = sorted(tokens_count, key = tokens_count.get, reverse = True)
for token in tokens_sorted:
    tokens_dict[token] = index
    index += 1

In [6]:
# build input file

with open(dataPath + 'cnn_token_dictionary.pkl', 'wb') as f:
    pickle.dump(tokens_dict, f)

NameError: name 'dataPath' is not defined

## Images data process
目前只將圖片載入成sparse matrix，生成im2latex_feature.train.pkl、im2latex_feature.validate.pkl與im2latex_feature.test.pkl。

In [ ]:
# write image pkl
# image shape: (1654,2339)

files = ['validate', 'test', 'train']

for fname in files:
    print fname 
    f = open(dataPath + 'im2latex_' + fname + '.lst')
    counter = 1
    
    imageList = [i.split()[1] + '.png' for i in f]
    numImage = len(imageList)

    image = np.rollaxis(scipy.misc.imread(ImagesPath+imageList[0], flatten = True),1)
    inverseImage = 255 - image
    reshapeImage = np.reshape(inverseImage,(1,-1))
    sparseImage = scipy.sparse.csr_matrix(reshapeImage)
    result = sparseImage

    for i in range(1,numImage):
        image = np.rollaxis(scipy.misc.imread(ImagesPath+imageList[i], flatten = True),1)
        inverseImage = 255 - image
        reshapeImage = np.reshape(inverseImage,(1,-1))
        sparseImage = scipy.sparse.csr_matrix(reshapeImage)
        
        counter += 1
        stdout.write("\r%d" % counter)
        stdout.flush()
        result = scipy.sparse.vstack((result, sparseImage))
        
    print result.shape
    
    resultSave32 = result.astype('float32')
    fileName = open(dataPath + 'im2latex_feature.' + fname + '.pkl','wb')
    pickle.dump(resultSave32, fileName ,-1)
    fileName.close()

## Align image with label
將label與image對齊，生成cnn_align.train.pkl、cnn_align.validate.pkl、cnn_align.test.pkl。
要用時，先載入label list，再載入image list (list of sparse matrix)。

In [15]:
# write align pkl

files = ['validate', 'test', 'train']

for name in files:
    counter = 0
    images = pickle.load(open(DATA_PATH + 'im2latex_feature.' + name +'.pkl','rb'))
    examples = open(DATA_PATH + 'im2latex_' + name + '.lst')
    labels = []
    for example in examples:
        label = formula_tokens_list[int(example.split()[0])]
        labels.append(label)
        counter += 1
    saveFile = open(DATA_PATH + 'cnn_align.' + name + '.pkl', 'wb')
    pickle.dump(labels, saveFile, protocol=pickle.HIGHEST_PROTOCOL)
    pickle.dump(images, saveFile, protocol=pickle.HIGHEST_PROTOCOL)
    saveFile.close()

# Some test

In [ ]:
tokens_dict['=']

In [ ]:
tokens_count

In [ ]:
tokens_sorted

In [ ]:
formula_tokens_list[1]